# 基于大语言模型的题库生成器

### 1. 获取 API key 并创建调用 API 的函数

首先使用`pip install dashscope`安装阿里云百炼的 Python SDK

然后导入相关依赖包，使用`getpass`输入api_key

In [12]:
from getpass import getpass
from dashscope import Generation
import os
# 安全获取API Key（输入时不显示明文）
# api_key = getpass("请输入阿里云百炼API Key: ")
api_key = "sk-c64994ca9f9741888dc7bccdef2aa0a1"
os.environ["DASHSCOPE_API_KEY"] = api_key
Generation.api_key = api_key


### 2. 通过关键词检索知识库，并获取相应的文段

使用`pip install pypdf faiss-cpu`安装依赖包

#### 2.1 初步创建向量数据库，根据所选文件进行切分和向量化
#### 2.2 加载已有的向量数据库并添加新文档
#### 2.3 根据关键词检索数据库

In [12]:
### 完整RAG流程测试
# import RAG_vector_store

# RAG_vector_store.create_vector_store("./data/sample.pdf")
# RAG_vector_store.add_new_documents_to_vector_store("./data/alibaba-2.txt")
# results = RAG_vector_store.search_similar_documents("延长学制", k=5)
# results2 = RAG_vector_store.search_similar_documents("阿里", k=5)

# print("------------------------------")
# print(results)

In [20]:
import LLM_functions
# 测试调用

keyword = "阿里"
result = LLM_functions.generate_QA(keyword=keyword, nums=5, model="qwen-plus", api_key=os.environ.get("DASHSCOPE_API_KEY"))

查询: 阿里
找到 5 个相关文档:

--- 文档 1 ---
内容: 我们坚持开源发展的承诺，让通义千问第二代开源模型获得全球开发者的广泛认可。通义千问第二代开源模型在推理、代码生成和数学等领域展现显著的性能提升，同时增强安全功能，扩展至支持27种语言。使用阿里云AI平台（百炼）的付费用户数量环比增长超过200%。

 

2024年巴黎奥运会上，阿里云的技术通过云基础设施实现远程视频制作和传输，在奥运史上首次代替卫星成为主要的传输方式。三分之二的转播商使用阿里云在...
来源: ./data/alibaba-2.txt
页码: N/A
--------------------------------------------------
--- 文档 2 ---
内容: 本地生活集团

 

截至2024年6月30日止季度，本地生活集团收入同比增长12%至人民币162.29亿元（22.33亿美元），由高德和饿了么订单增长，以及市场营销服务收入增长所带动。本季度，受惠于运营效率改善和业务规模提升，本地生活集团亏损同比大幅收窄。

 

大文娱集团

 

截至2024年6月30日止季度，大文娱集团的收入为人民币55.81亿元（7.68亿美元），同比增长4%，主要由其...
来源: ./data/alibaba-2.txt
页码: N/A
--------------------------------------------------
--- 文档 3 ---
内容: 我们通过在例如推荐和匹配算法等技术上的投入，以及通过由多元化供应商提供更多有价格竞争力的供给，改善了产品与用户需求的匹配度，提升了用户流量转化为购买力。今年4月，我们推出一款新的AI驱动的全平台市场营销工具“全站推广”，具备自动出价、优化目标人群定位和效果看板可视化功能。该新产品旨在提高商家市场营销投入及改善其市场营销效率，我们看到使用此产品的商家数稳步提升。



本季度，我们实现线上GMV同比...
来源: ./data/alibaba-2.txt
页码: N/A
--------------------------------------------------
--- 文档 4 ---
内容: 阿里巴巴集团公布2024年6月份季度业绩
2024年8月15日
中国杭州，2024年8月15日 

In [22]:
### 测试：根据题库答案生成干扰选项
print(result[3])
print(type(result[3]))
LLM_functions.generate_options(
    answer=str(result[3]),
)

LLM_functions.generate_fill_in_the_blank(
    answer=str(result[3]),
)

{'Q': '速卖通平台与哪家巴西领先零售商建立了合作关系？', 'A': 'Magazine Luiza（“Magalu”）。'}
<class 'dict'>


'速卖通平台与巴西领先零售商（ ）建立了合作关系。'

### 3. 将生成的题库保存到本地

In [5]:
import question_list

# 调用函数添加question_list
question_list.add_question_list(
    existing_file_path="./question_list/question_lists.json", 
    new_list_name=keyword,
    new_qa_list=result
)

已成功添加阿里，与原有列表并列


### 4. 后端接口设计

- 导入本地文件
  - .txt
  - .md? // todo
  - .pdf
- 根据主题检索知识库文段
- 文段导入大模型生成QA问答
  - 单个文段导入大模型
  - 循环生成
- 处理生成的QA问答实现数据格式化
- 保存生成的题库至本地


### 5. 使用 gradio 实现前端界面

In [6]:
import gradio as gr

# 创建Gradio聊天界面
demo = gr.ChatInterface(
    fn=LLM_functions.call_qwen,  # 传入函数对象，而非调用结果
    title="通义千问对话助手",
    description="与通义千问模型进行对话，支持多轮交流"
    
)

demo2 = gr.Interface(
    fn=LLM_functions.generate_QA,
    inputs=[
        gr.Textbox(label="关键词", placeholder="请输入关键词"),
        gr.Number(label="返回结果数量", value=5, step=1),
        gr.Textbox(label="模型名称", value="qwen-turbo", placeholder="请输入模型名称"),
    ],
    outputs=gr.JSON(label="生成的Q&A列表"),
    title="Q&A生成器",
    description="根据关键词生成相关的Q&A列表"
)

# 启动界面
if __name__ == "__main__":
    # demo.launch(share=True)
    demo2.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7343c3ffd2d4d83940.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
